# Preparation Script

### Setup

In [9]:
import wiki_dump_parser
from os import listdir, remove, system
from os.path import join
import subprocess
from multiprocessing import Pool
dataFolder="./data/"
backupDate="20200401"


### Download Files

In [19]:

packs = list(map(lambda x: x+1, range(27)))

for currentPack in packs:
    fileName = f"{dataFolder}/{currentPack}"
    #result = system(command, shell=True)
    subprocess.call(["wget","-0",f"{dataFolder}/{currentPack}.xml.gz","http://ftp.acc.umu.se/mirror/wikimedia.org/dumps/enwiki/{backupDate}/enwiki-{backupDate}-stub-meta-history{currentPack}.xml.gz"])

FileNotFoundError: [WinError 2] The system cannot find the file specified

### Dump to csv

First we need to convert xml dumps to csvs for further processing. The files should be residing in ```data/raw/xml```. Don't forget putting xml's under their respective folder, ex: ```data/raw/xml/en/enwiki-20200401-stub-meta-history1.xml```.

### Gather the filenames

In [3]:
rawXmlsPath = f"{dataFolder}/raw"
rawDataAtlas = {}

for langdir in listdir(rawXmlsPath):
    langdirPath = join(rawXmlsPath, langdir)
    rawDataAtlas[langdir] = []
    for xmlFile in listdir(langdirPath):
        xmlFilePath = join(langdirPath, xmlFile)
        rawDataAtlas[langdir].append(xmlFilePath)
# print(listdir(rawXmlsPath))

rawDataAtlas

{'en': ['./data/raw\\en\\enwiki-20200401-stub-meta-history1.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history10.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history11.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history12.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history13.xml',
  './data/raw\\en\\enwiki-20200401-stub-meta-history13.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history14.xml',
  './data/raw\\en\\enwiki-20200401-stub-meta-history14.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history15.xml',
  './data/raw\\en\\enwiki-20200401-stub-meta-history15.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history16.xml',
  './data/raw\\en\\enwiki-20200401-stub-meta-history17.xml',
  './data/raw\\en\\enwiki-20200401-stub-meta-history18.xml',
  './data/raw\\en\\enwiki-20200401-stub-meta-history19.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history2.xml.gz',
  './data/raw\\en\\enwiki-20200401-stub-meta-history20

### Convert the files to csvs

In [ ]:
def processXmlData(xmlDataFile):
    print(f"Processing {xmlDataFile}")
    wiki_dump_parser.xml_to_csv(xmlFileName)
    csvFileName = xmlFileName.replace(".xml",".csv")
    remove(xmlFileName)

    # Convert the csv to parquet for faster processing.
    csv_data = pd.read_csv(csvFileName, quotechar ='|')
    parquetFileName = csvFileName.replace(".csv", ".parquet")
    csv_data.to_parquet(parquetFileName, engine='pyarrow')
    remove(csvFileName)
    print(f"Completed Processing {xmlDataFile}!")


def processZip(zipPath):
    print(f"Extracting: {rawDataFile}")
    # result = subprocess.run(['./unpigz.exe', '-d', rawDataFile], capture_output=True)
    extractedFileName = rawDataFile.replace(".gz","")
    print(f"Extracted file: {rawDataFile}")
    processXmlData(extractedFileName)
    
for languageKey in rawDataAtlas:
    rawDataFiles = rawDataAtlas[languageKey]
    # print(rawDataFiles)
    processPool = Pool(6)
    processPool.map(processZip, rawDataFiles)    